# Benchmark m4

How to fit several model and evaluate its predictions on three of subsets of M4 data. 


#### functools.partial

Return a new `partial` object which when called will behave like *func* called with the positional arguments args and keyword arguments keywords. 

- [Source](https://docs.python.org/2/library/functools.html)

## Methods  

Brief overview of the used methods

### `DeepAREstimator`

- [`gluonts.model.deepar`](https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html)

Construct a DeepAR estimator, implements an RNN-based model, close to the one described in

- Salinas, David, Valentin Flunkert, and Jan Gasthaus. “DeepAR: Probabilistic forecasting with autoregressive recurrent networks.” arXiv preprint arXiv:1704.04110 (2017).


### `MQCNNEstimator`

- [`gluonts.model.seq2seq`](http://gluon-ts.mxnet.io/api/gluonts/gluonts.model.seq2seq.html)

An `MQDNNEstimator` with Convolutional Neural Network (CNN) as an encoder. Implements the MQ-CNN Forecaster, proposed in 

- Wen, Ruofeng, et al. “A multi-horizon quantile recurrent forecaster.” arXiv preprint arXiv:1711.11053 (2017).


### `SimpleFeedForwardEstimator`

- [`gluonts.model.simple_feedforward`](http://gluon-ts.mxnet.io/api/gluonts/gluonts.model.simple_feedforward.html)

A simple multilayer perceptron model predicting the next target time-steps given the previous ones. 



In [13]:
# install gluonts (most recent version)
!pip install git+https://github.com/awslabs/gluon-ts.git

  Cloning https://github.com/awslabs/gluon-ts.git to /tmp/pip-req-build-q5f9z9sx
  Installing build dependencies ... done
  Running setup.py bdist_wheel for gluonts ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-68t3gd2x/wheels/09/ba/b9/c830628df4d977a9dfe661f6001cdb861fc3f2524bac0ee433
Successfully built gluonts
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [14]:
# imports
import pandas as pd

import pprint
from functools import partial

# gluonts imports
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.distribution.piecewise_linear import PiecewiseLinearOutput
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.trainer import Trainer

# gluonts models
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.seq2seq import MQCNNEstimator
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator


In [15]:
# gluonts version (important)
import gluonts
print(gluonts.__version__)

0.3.4.dev81+g5ee5090


Number of series in m4 data: 

- Yearly: 23,000
- Quarterly: 24,000
- Monthly: 48,000
- Weekly: 359
- Daily: 4227
- Hourly: 414



In [16]:
epochs = 400
num_batches_per_epoch = 100

datasets = [
#     "m4_hourly",
#     "m4_daily",
#     "m4_weekly",
#     "m4_monthly",
#     "m4_quarterly",
    "m4_yearly",
]
print(datasets)

['m4_yearly']


In [17]:
estimators = [
#     partial(
#         SimpleFeedForwardEstimator,
#         trainer=Trainer(
#             epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
#         ),
#     ),
    partial(
        DeepAREstimator,
        trainer=Trainer(
            epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
        ),
    ),
#     partial(
#         DeepAREstimator,
#         distr_output=PiecewiseLinearOutput(8),
#         trainer=Trainer(
#             epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
#         ),
#     ),
#     partial(
#         MQCNNEstimator,
#         trainer=Trainer(
#             epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
#         ),
#     ),
]


INFO:root:Using CPU


In [18]:
def evaluate(dataset_name, estimator):
    dataset = get_dataset(dataset_name)
    estimator = estimator(
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True,
        cardinality=[
            feat_static_cat.cardinality
            for feat_static_cat in dataset.metadata.feat_static_cat
        ],
    )

    print(f"evaluating {estimator} on {dataset}")

    predictor = estimator.train(dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )

    pprint.pprint(agg_metrics)

    eval_dict = agg_metrics
    eval_dict["dataset"] = dataset_name
    eval_dict["estimator"] = type(estimator).__name__
    return eval_dict

In [19]:
datasets

['m4_yearly']

In [20]:
import numpy as np
import mxnet as mx

In [ ]:
if __name__ == "__main__":
    
    mx.random.seed(42)
    np.random.seed(42)
    
    results = []
    for dataset_name in datasets:
        for estimator in estimators:
            print(estimator)
            # catch exceptions that are happening during training to avoid failing the whole evaluation
            try:
                results.append(evaluate(dataset_name, estimator))
            except Exception as e:
                print(str(e))

    df = pd.DataFrame(results)

    sub_df = df[
        [
            "dataset",
            "estimator",
            "RMSE",
            "mean_wQuantileLoss",
            "MASE",
            "sMAPE",
            "MSIS",
        ]
    ]

    print(sub_df.to_string())

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_yearly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/100 [00:00<?, ?it/s]

functools.partial(<class 'gluonts.model.deepar._estimator.DeepAREstimator'>, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=None, epochs=400, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=100, patience=10, weight_decay=1e-08))
evaluating gluonts.model.deepar._estimator.DeepAREstimator(cardinality=[23000], cell_type="lstm", context_length=None, distr_output=gluonts.distribution.student_t.StudentTOutput(), dropout_rate=0.1, embedding_dimension=None, freq="12M", lags_seq=None, num_cells=40, num_layers=2, num_parallel_samples=100, prediction_length=6, scaling=True, time_features=None, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=None, epochs=400, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=100, patience=10, weight_decay=1e-08), use_feat_dynamic_real=Fa

INFO:root:Number of parameters in DeepARTrainingNetwork: 1179723
100%|██████████| 100/100 [00:04<00:00, 22.78it/s, avg_epoch_loss=8.14]
INFO:root:Epoch[0] Elapsed time 4.395 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=8.140023
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.46it/s, avg_epoch_loss=6.82]
INFO:root:Epoch[1] Elapsed time 4.268 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.816145
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.22it/s, avg_epoch_loss=6.89]
INFO:root:Epoch[2] Elapsed time 4.312 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.890125
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 23.60it/s, avg_epoch_loss=7.26]
INFO:root:Epoch[3] Elapsed time 4.243 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.256244
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 24.22it/s, avg_epoch_los

100%|██████████| 100/100 [00:09<00:00, 10.43it/s, avg_epoch_loss=5.56]
INFO:root:Epoch[37] Elapsed time 9.590 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=5.560893
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 100/100 [00:14<00:00,  7.05it/s, avg_epoch_loss=6.2]
INFO:root:Epoch[38] Elapsed time 14.217 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=6.198479
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 100/100 [00:14<00:00,  7.08it/s, avg_epoch_loss=6.52]
INFO:root:Epoch[39] Elapsed time 14.121 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=6.524188
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 100/100 [00:13<00:00,  7.34it/s, avg_epoch_loss=7]  
INFO:root:Epoch[40] Elapsed time 13.649 seconds
INFO:root:Epoch[40] Evaluation metric 'epoch_loss'=6.995295
INFO:root:Epoch[41] Learning rate is 0.001
100%|██████████| 100/100 [00:14<00:00,  7.00it/s, avg_epoch_loss=6.22]
INFO:root:Epoch[41] Elapsed time 14.308 seco

100%|██████████| 100/100 [00:13<00:00,  7.20it/s, avg_epoch_loss=5.54]
INFO:root:Epoch[73] Elapsed time 13.889 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.536370
INFO:root:Epoch[74] Learning rate is 0.000125
100%|██████████| 100/100 [00:14<00:00,  7.06it/s, avg_epoch_loss=6.32]
INFO:root:Epoch[74] Elapsed time 14.162 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=6.323503
INFO:root:Epoch[75] Learning rate is 0.000125
100%|██████████| 100/100 [00:13<00:00,  7.19it/s, avg_epoch_loss=6.58]
INFO:root:Epoch[75] Elapsed time 13.924 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=6.575580
INFO:root:Epoch[76] Learning rate is 0.000125
100%|██████████| 100/100 [00:13<00:00,  7.23it/s, avg_epoch_loss=6.82]
INFO:root:Epoch[76] Elapsed time 13.860 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=6.815035
INFO:root:Epoch[77] Learning rate is 0.000125
100%|██████████| 100/100 [00:14<00:00,  7.03it/s, avg_epoch_loss=6.25]
INFO:root:Epoch[77] Elapsed t

...

INFO:root:Epoch[218] Evaluation metric 'epoch_loss'=6.399652
INFO:root:Epoch[219] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 20.63it/s, avg_epoch_loss=6.77]
INFO:root:Epoch[219] Elapsed time 4.849 seconds
INFO:root:Epoch[219] Evaluation metric 'epoch_loss'=6.768792
INFO:root:Epoch[220] Learning rate is 5e-05
100%|██████████| 100/100 [00:08<00:00, 11.21it/s, avg_epoch_loss=6.3]
INFO:root:Epoch[220] Elapsed time 8.926 seconds
INFO:root:Epoch[220] Evaluation metric 'epoch_loss'=6.301478
INFO:root:Epoch[221] Learning rate is 5e-05
100%|██████████| 100/100 [00:14<00:00,  6.97it/s, avg_epoch_loss=6.54]
INFO:root:Epoch[221] Elapsed time 14.359 seconds
INFO:root:Epoch[221] Evaluation metric 'epoch_loss'=6.535864
INFO:root:Epoch[222] Learning rate is 5e-05
100%|██████████| 100/100 [00:13<00:00,  7.24it/s, avg_epoch_loss=6.2]
INFO:root:Epoch[222] Elapsed time 13.845 seconds
INFO:root:Epoch[222] Evaluation metric 'epoch_loss'=6.201358
INFO:root:Epoch[223] Learning rate is 5e-05

100%|██████████| 100/100 [00:15<00:00,  6.42it/s, avg_epoch_loss=6.8]
INFO:root:Epoch[255] Elapsed time 15.610 seconds
INFO:root:Epoch[255] Evaluation metric 'epoch_loss'=6.803537
INFO:root:Epoch[256] Learning rate is 5e-05
100%|██████████| 100/100 [00:14<00:00,  7.10it/s, avg_epoch_loss=6.18]
INFO:root:Epoch[256] Elapsed time 14.117 seconds
INFO:root:Epoch[256] Evaluation metric 'epoch_loss'=6.175149
INFO:root:Epoch[257] Learning rate is 5e-05
100%|██████████| 100/100 [00:13<00:00,  7.16it/s, avg_epoch_loss=6.47]
INFO:root:Epoch[257] Elapsed time 13.984 seconds
INFO:root:Epoch[257] Evaluation metric 'epoch_loss'=6.467534
INFO:root:Epoch[258] Learning rate is 5e-05
100%|██████████| 100/100 [00:14<00:00,  7.10it/s, avg_epoch_loss=6.21]
INFO:root:Epoch[258] Elapsed time 14.116 seconds
INFO:root:Epoch[258] Evaluation metric 'epoch_loss'=6.212136
INFO:root:Epoch[259] Learning rate is 5e-05
100%|██████████| 100/100 [00:13<00:00,  7.30it/s, avg_epoch_loss=5.7]
INFO:root:Epoch[259] Elapsed ti

INFO:root:Epoch[291] Evaluation metric 'epoch_loss'=6.807789
INFO:root:Epoch[292] Learning rate is 5e-05
100%|██████████| 100/100 [00:17<00:00,  5.81it/s, avg_epoch_loss=6.13]
INFO:root:Epoch[292] Elapsed time 17.242 seconds
INFO:root:Epoch[292] Evaluation metric 'epoch_loss'=6.130703
INFO:root:Epoch[293] Learning rate is 5e-05
100%|██████████| 100/100 [00:16<00:00,  6.01it/s, avg_epoch_loss=6.42]
INFO:root:Epoch[293] Elapsed time 16.694 seconds
INFO:root:Epoch[293] Evaluation metric 'epoch_loss'=6.423022
INFO:root:Epoch[294] Learning rate is 5e-05
100%|██████████| 100/100 [00:13<00:00,  7.22it/s, avg_epoch_loss=6.24]
INFO:root:Epoch[294] Elapsed time 13.871 seconds
INFO:root:Epoch[294] Evaluation metric 'epoch_loss'=6.238993
INFO:root:Epoch[295] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.23it/s, avg_epoch_loss=5.76]
INFO:root:Epoch[295] Elapsed time 4.309 seconds
INFO:root:Epoch[295] Evaluation metric 'epoch_loss'=5.760125
INFO:root:Epoch[296] Learning rate is 5e

100%|██████████| 100/100 [00:04<00:00, 22.83it/s, avg_epoch_loss=6.08]
INFO:root:Epoch[328] Elapsed time 4.385 seconds
INFO:root:Epoch[328] Evaluation metric 'epoch_loss'=6.080622
INFO:root:Epoch[329] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.39it/s, avg_epoch_loss=6.46]
INFO:root:Epoch[329] Elapsed time 4.278 seconds
INFO:root:Epoch[329] Evaluation metric 'epoch_loss'=6.457455
INFO:root:Epoch[330] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.02it/s, avg_epoch_loss=6.24]
INFO:root:Epoch[330] Elapsed time 4.349 seconds
INFO:root:Epoch[330] Evaluation metric 'epoch_loss'=6.243543
INFO:root:Epoch[331] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.90it/s, avg_epoch_loss=5.57]
INFO:root:Epoch[331] Elapsed time 4.185 seconds
INFO:root:Epoch[331] Evaluation metric 'epoch_loss'=5.568320
INFO:root:Epoch[332] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 22.96it/s, avg_epoch_loss=5.93]
INFO:root:Epoch[332] Elapsed time

INFO:root:Epoch[364] Evaluation metric 'epoch_loss'=6.188606
INFO:root:Epoch[365] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.20it/s, avg_epoch_loss=6.44]
INFO:root:Epoch[365] Elapsed time 4.315 seconds
INFO:root:Epoch[365] Evaluation metric 'epoch_loss'=6.436929
INFO:root:Epoch[366] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.07it/s, avg_epoch_loss=6.2]
INFO:root:Epoch[366] Elapsed time 4.336 seconds
INFO:root:Epoch[366] Evaluation metric 'epoch_loss'=6.196692
INFO:root:Epoch[367] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 23.54it/s, avg_epoch_loss=5.79]
INFO:root:Epoch[367] Elapsed time 4.254 seconds
INFO:root:Epoch[367] Evaluation metric 'epoch_loss'=5.792814
INFO:root:Epoch[368] Learning rate is 5e-05
100%|██████████| 100/100 [00:04<00:00, 21.91it/s, avg_epoch_loss=5.68]
INFO:root:Epoch[368] Elapsed time 4.567 seconds
INFO:root:Epoch[368] Evaluation metric 'epoch_loss'=5.678272
INFO:root:Epoch[369] Learning rate is 5e-05


In [ ]:
df[["dataset", "estimator", "MASE", "sMAPE", "MSIS","wQuantileLoss[0.5]", "wQuantileLoss[0.9]", ]]